# This file is for testing the camera calibraiont pipeline

In [7]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from pathlib import Path

In [8]:
# Cell 2 - Configuration
square_size = 0.1  # meters

chessboard_sizes = {
    'board1': (11, 7),
    'board2': (5, 7),
    'board3': (7, 5),
}

# All boards have the same square size
square_sizes = {name: square_size for name in chessboard_sizes}

# Termination criteria for sub-pixel refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points for each board type
objp_templates = {}
for name, size in chessboard_sizes.items():
    objp = np.zeros((size[0] * size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:size[0], 0:size[1]].T.reshape(-1, 2) * square_sizes[name]
    objp_templates[name] = objp

# Storage for calibration data
all_objpoints = []
all_imgpoints_left = []
all_imgpoints_right = []

# Create output directory for logging
output_dir = Path("calibration_debug")
output_dir.mkdir(exist_ok=True)

print("Configuration loaded successfully") 

Configuration loaded successfully


In [9]:
# Cell 3 - Load images
raw_path = "../data/34759_final_project_raw/"
left_raw_path = raw_path + "calib/image_02/data/"
right_raw_path = raw_path + "calib/image_03/data/"

left_images = sorted(glob.glob(left_raw_path + '*.png'))
right_images = sorted(glob.glob(right_raw_path + '*.png'))

if len(left_images) != len(right_images):
    print(f"Error: Mismatch! {len(left_images)} left vs {len(right_images)} right")
else:
    print(f"Successfully loaded {len(left_images)} image pairs")

Successfully loaded 19 image pairs


In [16]:
# Cell 4 - Detection function with better logic
def detect_multiple_chessboards(gray_img, img_color, chessboard_sizes, 
                                 objp_templates, criteria, 
                                 side_name="", img_idx=0, save_debug=True):
    """
    Detect multiple chessboards in a single image.
    Returns list of object points and corner points.
    """
    objpoints_found = []
    corners_found = []
    boards_info = []
    
    # Create a mask to black out already-found boards
    mask = np.ones_like(gray_img) * 255
    debug_img = img_color.copy()
    
    # Try each board size
    for board_name, board_size in chessboard_sizes.items():
        max_attempts = 10  # Prevent infinite loops
        attempts = 0
        
        while attempts < max_attempts:
            attempts += 1
            
            # Apply mask to ignore already-found regions
            masked_gray = cv2.bitwise_and(gray_img, mask)
            
            # Detect chessboard
            flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            ret, corners = cv2.findChessboardCornersSB(
                masked_gray, board_size, flags
            )
            
            if ret and corners is not None:
                # Refine corners for better accuracy
                #corners_refined = cv2.cornerSubPix(
                #    gray_img, corners, (11, 11), (-1, -1), criteria
                #)
                corners_refined = corners
                
                objpoints_found.append(objp_templates[board_name])
                corners_found.append(corners_refined)
                
                # Get bounding box and expand it slightly
                x_coords = corners_refined[:, 0, 0]
                y_coords = corners_refined[:, 0, 1]
                
                x_min, x_max = int(x_coords.min()), int(x_coords.max())
                y_min, y_max = int(y_coords.min()), int(y_coords.max())
                
                # Expand by 10% to ensure we mask the whole board
                margin_x = int((x_max - x_min) * 0.1)
                margin_y = int((y_max - y_min) * 0.1)
                
                x_min = max(0, x_min - margin_x)
                x_max = min(mask.shape[1], x_max + margin_x)
                y_min = max(0, y_min - margin_y)
                y_max = min(mask.shape[0], y_max + margin_y)
                
                # Black out this region in the mask
                mask[y_min:y_max, x_min:x_max] = 0
                
                # Draw on debug image
                cv2.drawChessboardCorners(debug_img, board_size, 
                                         corners_refined, True)
                cv2.rectangle(debug_img, (x_min, y_min), (x_max, y_max),
                            (0, 255, 0), 3)
                
                boards_info.append({
                    'name': board_name,
                    'size': board_size,
                    'bbox': (x_min, y_min, x_max, y_max)
                })
                
                print(f"  ✓ Found {board_name} {board_size}")
            else:
                # No more boards of this size
                break
    
    # Save debug image
    if save_debug and boards_info:
        output_path = output_dir / f"img_{img_idx:03d}_{side_name}.jpg"
        cv2.imwrite(str(output_path), debug_img)
    
    return objpoints_found, corners_found, boards_info, debug_img

In [17]:
# Cell 5 - Main detection loop with visualization
print("Starting calibration image processing...")
print("=" * 60)

successful_pairs = 0
failed_pairs = 0

for i in range(len(left_images)):
    print(f"\nProcessing pair {i+1}/{len(left_images)}: {Path(left_images[i]).name}")
    
    # Load images
    img_left = cv2.imread(left_images[i])
    img_right = cv2.imread(right_images[i])
    
    if img_left is None or img_right is None:
        print(f"  ✗ Failed to load images")
        failed_pairs += 1
        continue
    
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)
    
    # Detect boards in both images
    objpoints_left, corners_left, info_left, debug_left = detect_multiple_chessboards(
        gray_left, img_left, chessboard_sizes, objp_templates, 
        criteria, "left", i, save_debug=True
    )
    
    objpoints_right, corners_right, info_right, debug_right = detect_multiple_chessboards(
        gray_right, img_right, chessboard_sizes, objp_templates,
        criteria, "right", i, save_debug=True
    )
    
    # Must find same number of boards in both images
    if len(objpoints_left) != len(objpoints_right) or len(objpoints_left) == 0:
        print(f"  ✗ Board count mismatch: {len(objpoints_left)} left vs {len(objpoints_right)} right")
        failed_pairs += 1
        
        # Save comparison image
        comparison = np.hstack([debug_left, debug_right])
        cv2.imwrite(str(output_dir / f"FAILED_pair_{i:03d}.jpg"), comparison)
        continue
    
    # Check that boards match (same sizes)
    sizes_match = all(
        info_left[j]['size'] == info_right[j]['size'] 
        for j in range(len(info_left))
    )
    
    if not sizes_match:
        print(f"  ✗ Board sizes don't match between left and right")
        failed_pairs += 1
        continue
    
    # Success! Concatenate all boards found in this image pair
    all_objpoints.append(np.vstack(objpoints_left))
    all_imgpoints_left.append(np.vstack(corners_left))
    all_imgpoints_right.append(np.vstack(corners_right))
    
    successful_pairs += 1
    print(f"  ✓ SUCCESS: {len(objpoints_left)} boards detected and matched")
    
    # Save side-by-side comparison
    comparison = np.hstack([debug_left, debug_right])
    cv2.imwrite(str(output_dir / f"SUCCESS_pair_{i:03d}.jpg"), comparison)

print("\n" + "=" * 60)
print(f"Processing complete!")
print(f"  Successful pairs: {successful_pairs}")
print(f"  Failed pairs: {failed_pairs}")
print(f"  Total corner points: {sum(pts.shape[0] for pts in all_objpoints)}")
print(f"\nDebug images saved to: {output_dir.absolute()}")

Starting calibration image processing...

Processing pair 1/19: 0000000000.png
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✗ Board count mismatch: 2 left vs 7 right

Processing pair 2/19: 0000000001.png
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✗ Board count mismatch: 2 left vs 7 right

Processing pair 3/19: 0000000002.png
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✓ Found board2 (5, 7)
  ✗ Board count mismatch: 2 left vs 7 right

Processing pair 4/19: 0000000003.png
  ✓ Found board2 (5, 7)
  ✓

In [ ]:
# Cell 6 - Individual camera calibration
if len(all_objpoints) < 3:
    print("ERROR: Not enough successful calibration images!")
    print("Need at least 3, got", len(all_objpoints))
else:
    img_shape = gray_left.shape[::-1]  # (width, height)
    
    print("Calibrating left camera...")
    ret_l, mtx_l, dist_l, rvecs_l, tvecs_l = cv2.calibrateCamera(
        all_objpoints, all_imgpoints_left, img_shape, None, None
    )
    print(f"  RMS error: {ret_l:.4f}")
    
    print("Calibrating right camera...")
    ret_r, mtx_r, dist_r, rvecs_r, tvecs_r = cv2.calibrateCamera(
        all_objpoints, all_imgpoints_right, img_shape, None, None
    )
    print(f"  RMS error: {ret_r:.4f}")
    
    # Good calibration typically has RMS < 1.0 pixel
    if ret_l > 1.0 or ret_r > 1.0:
        print("\n⚠ WARNING: High RMS errors detected!")
        print("  This suggests problems with detection or board measurements")

In [ ]:
# Cell 7 - Stereo calibration
print("\nPerforming stereo calibration...")

stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
flags = cv2.CALIB_FIX_INTRINSIC  # Use individual calibrations

ret_stereo, mtx_l, dist_l, mtx_r, dist_r, R, T, E, F = cv2.stereoCalibrate(
    all_objpoints,
    all_imgpoints_left,
    all_imgpoints_right,
    mtx_l, dist_l,
    mtx_r, dist_r,
    img_shape,
    criteria=stereocalib_criteria,
    flags=flags
)

print(f"Stereo RMS error: {ret_stereo:.4f}")
print(f"\nBaseline (translation): {np.linalg.norm(T):.4f} m")
print(f"Translation vector: {T.flatten()}")

if ret_stereo > 1.0:
    print("\n⚠ WARNING: High stereo RMS error!")

In [ ]:
# Cell 8 - Stereo rectification
print("Computing rectification maps...")

R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(
    mtx_l, dist_l,
    mtx_r, dist_r,
    img_shape,
    R, T,
    alpha=0,  # 0=crop to valid pixels, 1=keep all pixels
    newImageSize=img_shape
)

map_left_x, map_left_y = cv2.initUndistortRectifyMap(
    mtx_l, dist_l, R1, P1, img_shape, cv2.CV_32FC1
)

map_right_x, map_right_y = cv2.initUndistortRectifyMap(
    mtx_r, dist_r, R2, P2, img_shape, cv2.CV_32FC1
)

print(f"Left ROI: {roi_left}")
print(f"Right ROI: {roi_right}")

def rectify_stereo_pair(img_left, img_right):
    """Apply rectification to a stereo pair."""
    rect_left = cv2.remap(img_left, map_left_x, map_left_y, cv2.INTER_LINEAR)
    rect_right = cv2.remap(img_right, map_right_x, map_right_y, cv2.INTER_LINEAR)
    return rect_left, rect_right

print("Rectification ready!")

In [ ]:
# Cell 9 - Test rectification on a sample image
test_idx = 0
img_left_test = cv2.imread(left_images[test_idx])
img_right_test = cv2.imread(right_images[test_idx])

rect_left, rect_right = rectify_stereo_pair(img_left_test, img_right_test)

# Draw horizontal lines to verify alignment
def draw_horizontal_lines(img, num_lines=20):
    img_lines = img.copy()
    h, w = img.shape[:2]
    for i in range(num_lines):
        y = int(h * i / num_lines)
        cv2.line(img_lines, (0, y), (w, y), (0, 255, 0), 1)
    return img_lines

rect_left_lines = draw_horizontal_lines(rect_left)
rect_right_lines = draw_horizontal_lines(rect_right)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes[0, 0].imshow(cv2.cvtColor(img_left_test, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Left')
axes[0, 1].imshow(cv2.cvtColor(img_right_test, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('Original Right')
axes[1, 0].imshow(cv2.cvtColor(rect_left_lines, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Rectified Left (with epipolar lines)')
axes[1, 1].imshow(cv2.cvtColor(rect_right_lines, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title('Rectified Right (with epipolar lines)')

for ax in axes.flat:
    ax.axis('off')
plt.tight_layout()
plt.savefig(output_dir / 'rectification_test.jpg', dpi=150, bbox_inches='tight')
plt.show()

print("If rectification is correct, horizontal lines should align across left/right images")

## Exercise 1
The process of calibrating an image consists of mainly 3 steps: 1) find chessboard-corners in a dataset of images containing a chessboard. 2) Use the corner points to compute a camera matrix. 3) Use the camera matrix to undistort images.

After setting some optimization parameters we can loop over all the images in the `imgs` folder and extract the checkerboard corners.

Use any of the images in the folder `imgs` to extract the number of checkerboard corners there are on the checkerboard. Fill in the information in `nb_vertical` and `nb_horizontal` and look up the opencv [findChessboardCorners](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a) function and implement it in the below code snippet .

In [2]:
# Define the chessboard dimensions for *each* type of chessboard you are using
# If you have multiple identical chessboards, you only need one definition.

square_size = 0.1 # m

chessboard_sizes = {
    'board1': (11, 7),  # eg big one on the left
    'board2': (5, 7), #  eg smaller ones in the middle for example
    'board3': (7, 5), # eg same as the prev one but on the side
}
square_sizes = {
    'board1': square_size, # in your chosen unit
    'board2': square_size,
    'board3': square_size,
}

# Termination criteria for sub-pixel accuracy
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points for each board type
objp_templates = {}
for name, size in chessboard_sizes.items():
    objp = np.zeros((size[0] * size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:size[0], 0:size[1]].T.reshape(-1, 2) * square_sizes[name]
    objp_templates[name] = objp

#print(objp_templates)

# Arrays to store object points and image points from all the images.
# These will store detections from ALL boards found in a single image pair.
all_objpoints = []
all_imgpoints_left = []
all_imgpoints_right = []

# Get lists of image paths
# Make sure your image pairs are correctly matched (e.g., left_001.jpg, right_001.jpg)
raw_path = "../data/34759_final_project_raw/"
left_raw_path = raw_path + "calib/image_02/data/"
right_raw_path = raw_path + "calib/image_03/data/"

left_images = sorted(glob.glob(left_raw_path + '*.png'))
right_images = sorted(glob.glob(right_raw_path + '*.png'))



if len(left_images) != len(right_images):
    print("Error: Number of left and right images do not match!")
else:
    print("Image loading success")
    #print(len(left_images))

Image loading success


In [3]:
iteration = 0
logging_img = cv2.imread(left_images[0])
logging_img = cv2.cvtColor(logging_img, cv2.COLOR_BGR2GRAY) 

#logging_img_corners = cv2.imread(left_images[0])

logging_left = np.ndarray(logging_img.shape)
logging_right = np.ndarray(logging_img.shape)

#fig, ax = plt.subplots(nrows=len(left_images), ncols=1, figsize=(12, len(left_images) * 12))
#ax = ax.ravel()

for i in range(len(left_images) - 0):
    # loop over every image
    img_left = cv2.imread(left_images[i])
    img_right = cv2.imread(right_images[i])
            
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY) 
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    image_objpoints = []
    image_corners_left = []
    image_corners_right = []

    for name, size in chessboard_sizes.items():
        # iterating through the various chessboard sizes
        do_while = 1
        repeat = False
        while do_while == 1 or repeat == True:
            do_while = 0
            # loop ever every single chessboard of the same size
            #print(f"%s found", name)

            h_l, w_l = gray_left.shape
            h_r, w_r = gray_right.shape

            manual_filter_start = (700, 300)
            manual_filter_stop = (900, 400)
            #manual_filter_start = (100, 200)
            #manual_filter_stop = (200, 300)
            
            #gray_left = cv2.rectangle(gray_left, manual_filter_start, manual_filter_stop, (0, 0, 0), -1)
            #gray_right = cv2.rectangle(gray_right, manual_filter_start, manual_filter_stop, (0, 0, 0), -1)

            #logging_img = cv2.rectangle(logging_img, manual_filter_start, manual_filter_stop, (0, 0, 0), -1)


            flags = 0#cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE | cv2.CALIB_CB_FILTER_QUADS
            detected_boards_left, corners_left = cv2.findChessboardCornersSB(gray_left, size, flags)
            detected_boards_right, corners_right = cv2.findChessboardCornersSB(gray_right, size, flags)


            iteration += 1
            if detected_boards_left:
                #print(f"detected on the left, %d", iteration)
                pass
            
            if detected_boards_left and detected_boards_right and corners_left.shape == corners_right.shape:
                repeat = True
                #print("detection")
                #repeat = False

                # Refine corners
                #corners_left = cv2.cornerSubPix(gray_left, corners_left, (11, 11), (-1, -1), criteria)
                #corners_right = cv2.cornerSubPix(gray_right, corners_right, (11, 11), (-1, -1), criteria)


                image_objpoints.append(objp_templates[name])
                image_corners_left.append(corners_left)
                image_corners_right.append(corners_right)

                # cut the already detected chessboard out of the image
                corner_left_expanded = ((corners_left[0][0]).astype(int), (corners_left[-1][0]).astype(int))
                corner_right_expanded = ((corners_right[0][0]).astype(int), (corners_right[-1][0]).astype(int))

                gray_left = cv2.rectangle(gray_left, corner_left_expanded[0], corner_left_expanded[1], (0, 0, 0), -1)
                gray_right = cv2.rectangle(gray_right, corner_right_expanded[0], corner_right_expanded[1], (0, 0, 0), -1)

                logging_left = cv2.drawChessboardCorners(gray_left, size, corners_left, True)
                logging_right = cv2.drawChessboardCorners(gray_right, size, corners_right, True)

                #logging_img = cv2.rectangle(gray_left, corner_left_expanded[0], corner_left_expanded[1], (0, 0, 0), -1)
                #logging_img_corners = cv2.drawChessboardCorners(logging_img_corners, size, corners_left, True)

            else:
                repeat = False

    if image_objpoints:
                # Concatenate all boards found in this image
        all_objpoints.append(np.vstack(image_objpoints))
        all_imgpoints_left.append(np.vstack(image_corners_left))
        all_imgpoints_right.append(np.vstack(image_corners_right)) 

        cv2.imwrite('left_' + str(i) + ".jpg", logging_left)
        cv2.imwrite('right_' + str(i) + ".jpg", logging_right)

        #ax[i] = plt.imshow(gray_left)
        
print(iteration)
#fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(14, 14))
#ax[0].imshow(logging_img, cmap='gray')
#ax[1].imshow(logging_img_corners)
#plt.show()

93


In [4]:
img_shape_img = cv2.imread(left_images[0])
img_shape_img = cv2.cvtColor(img_shape_img, cv2.COLOR_BGR2GRAY) 

img_shape = img_shape_img.shape
img_shape = (img_shape[1], img_shape[0])

ret_l, mtx_l, dist_l, rvecs_l, tvecs_l = cv2.calibrateCamera(
    all_objpoints, 
    all_imgpoints_left, 
    img_shape, 
    None, 
    None
)
print(f"Left camera RMS reprojection error: {ret_l:.4f}")

ret_r, mtx_r, dist_r, rvecs_r, tvecs_r = cv2.calibrateCamera(
    all_objpoints, 
    all_imgpoints_right, 
    img_shape, 
    None, 
    None
)
print(f"Right camera RMS reprojection error: {ret_r:.4f}")

# Stereo calibration
stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)

flags = cv2.CALIB_FIX_INTRINSIC  # Use individual calibrations as initial estimates
# Alternative: flags = 0 to refine everything

ret_stereo, mtx_l, dist_l, mtx_r, dist_r, R, T, E, F = cv2.stereoCalibrate(
    all_objpoints,
    all_imgpoints_left,
    all_imgpoints_right,
    mtx_l, dist_l,
    mtx_r, dist_r,
    img_shape,
    criteria=stereocalib_criteria,
    flags=flags
)

print(f"Stereo calibration RMS error: {ret_stereo:.4f}")
print(f"\nRotation matrix:\n{R}")
print(f"\nTranslation vector (baseline):\n{T.flatten()}")
print(f"Baseline distance: {np.linalg.norm(T):.4f} m")

Left camera RMS reprojection error: 92.4372
Right camera RMS reprojection error: 88.5031
Stereo calibration RMS error: 3945.1013

Rotation matrix:
[[ 0.2281529   0.56253205 -0.79467223]
 [-0.68209411  0.67477463  0.28182765]
 [ 0.69476174  0.47774145  0.53765159]]

Translation vector (baseline):
[0.34230262 0.42577316 0.24155909]
Baseline distance: 0.5973 m


In [5]:
R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(
    mtx_l, dist_l,
    mtx_r, dist_r,
    img_shape,
    R, T,
    alpha=0,  # 0 = crop to valid pixels only, 1 = keep all pixels
    newImageSize=img_shape
)

map_left_x, map_left_y = cv2.initUndistortRectifyMap(
    mtx_l, dist_l, R1, P1, img_shape, cv2.CV_32FC1
)

map_right_x, map_right_y = cv2.initUndistortRectifyMap(
    mtx_r, dist_r, R2, P2, img_shape, cv2.CV_32FC1
)

print(f"Left ROI: {roi_left}")
print(f"Right ROI: {roi_right}")

Left ROI: (0, 0, 0, 0)
Right ROI: (0, 0, 0, 0)


In [6]:
def rectify_stereo_pair(img_left: np.ndarray, img_right: np.ndarray):

    """Apply rectification to a stereo pair."""
    rect_left = cv2.remap(img_left, map_left_x, map_left_y, cv2.INTER_LINEAR)
    rect_right = cv2.remap(img_right, map_right_x, map_right_y, cv2.INTER_LINEAR)
    return rect_left, rect_right

In [ ]:
img = cv2.imread(left_images[0])
rect_img, _ = rectify_stereo_pair(img, img)

plt.imshow(rect_img)
#fig, ax = plt.subfigures(nrows=2, ncols=1, figsize=(14, 14))
#ax[0] = plt.imshow()

In [ ]:
'''
gray_img = cv2.imread(left_images[0])
gray_img = cv2.cvtColor(gray_img, cv2.COLOR_BGR2GRAY)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(all_objpoints, all_imgpoints_left, gray_img.shape[::-1], None, None)
img = cv2.imread(left_images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
'''

In [ ]:
'''
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(18,18))
ax[0].imshow(img[...,[2,1,0]])
ax[0].set_title('Original image')
ax[1].imshow(dst[...,[2,1,0]])
ax[1].set_title('Undistorted image')
'''

In [ ]:
'''
# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
plt.figure(figsize=(10,10))
plt.imshow(dst[...,[2,1,0]])
'''